<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from zipfile import ZipFile, Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import voila
import streamlit as st
import statsmodels.api as sm
import plotly.graph_objects as go
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from functions import *

import warnings
warnings.filterwarnings('ignore')

def f(x):
    return x

interact(f, x=10)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.f(x)>

In [ ]:
data_load = pd.read_csv("./data/PGclean_data.csv",low_memory=False) #index_col="product_name"
print("Voici le nouveau remplissage du jeu de données avec toutes les données imputées (à l\'exception du nutriscore) :")
data_load.head(10)